In [1]:
import pre_process as pp
import utils as u
import dataset as dt
import numpy as np
import pandas as pd
import joblib as jb
from tqdm import tqdm
from collections import Counter
import time
import os

In [2]:
path_list_tr = u.get_path_list("D:/isip.piconepress.com/projects/nedc/data/tuh_eeg/tuh_eeg_abnormal/v3.0.1/edf/train",['.edf'],True)
path_list_eval = u.get_path_list("D:/isip.piconepress.com/projects/nedc/data/tuh_eeg/tuh_eeg_abnormal/v3.0.1/edf/eval",['.edf'],True)

In [3]:
print(type(path_list_tr))
print(path_list_tr.shape)
print(type(path_list_eval))
print(path_list_eval.shape)

<class 'numpy.ndarray'>
(2717,)
<class 'numpy.ndarray'>
(276,)


In [7]:
path_id_list_tr= [f"campione_{i}_tr" for i in range(1, 2718)]
path_id_list_eval= [f"campione_{i}_eval" for i in range(1, 277)]

In [8]:
df_path_tr = pd.DataFrame({'Path_id':path_id_list_tr, 'Path':path_list_tr})
df_path_eval = pd.DataFrame({'Path_id':path_id_list_eval, 'Path':path_list_eval})

In [9]:
df_path_tr.tail()

,Path_id,Path
2712,campione_2713_tr,D:\isip.piconepress.com\projects\nedc\data\tuh...
2713,campione_2714_tr,D:\isip.piconepress.com\projects\nedc\data\tuh...
2714,campione_2715_tr,D:\isip.piconepress.com\projects\nedc\data\tuh...
2715,campione_2716_tr,D:\isip.piconepress.com\projects\nedc\data\tuh...
2716,campione_2717_tr,D:\isip.piconepress.com\projects\nedc\data\tuh...


In [10]:
df_path_eval.tail()

,Path_id,Path
271,campione_272_eval,D:\isip.piconepress.com\projects\nedc\data\tuh...
272,campione_273_eval,D:\isip.piconepress.com\projects\nedc\data\tuh...
273,campione_274_eval,D:\isip.piconepress.com\projects\nedc\data\tuh...
274,campione_275_eval,D:\isip.piconepress.com\projects\nedc\data\tuh...
275,campione_276_eval,D:\isip.piconepress.com\projects\nedc\data\tuh...


In [14]:
df_path_tr.to_csv('path_tr.csv', index=False, encoding='utf-8-sig')
df_path_eval.to_csv('path_eval.csv', index=False, encoding='utf-8-sig')

In [2]:
def worker(in_file, out_dir, out_prefix):
    try:
        dg = pp.data_gen(in_file, out_dir, out_prefix)
        dg.save_final_data(seg_len=5.0, merge_len=1)
    except:
        flag = False
        time.sleep(3.0)
    else:
        flag = True
    return in_file, flag

In [3]:
md_file_tr = "path_tr.csv"
out_dir_tr = "D:/raw_data_tr"
log_file_tr = "D:/raw_data_tr/log_tr.csv"
md_file_eval = "path_eval.csv"
out_dir_eval = "D:/raw_data_eval"
log_file_eval = "D:/raw_data_eval/log_tr.csv"

In [4]:
if not os.path.isdir(out_dir_tr):
    os.makedirs(out_dir_tr)

df_path_tr = pd.read_csv(md_file_tr)
tasks_tr = [(Pt, Pt_id) for Pt, Pt_id in zip(df_path_tr.Path, df_path_tr.Path_id)]
tasks_tr[0]

('D:\\isip.piconepress.com\\projects\\nedc\\data\\tuh_eeg\\tuh_eeg_abnormal\\v3.0.1\\edf\\train\\abnormal\\01_tcp_ar\\aaaaaaaq_s004_t000.edf',
 'campione_1_tr')

In [5]:
if not os.path.isdir(out_dir_eval):
    os.makedirs(out_dir_eval)

df_path_eval = pd.read_csv(md_file_eval)
tasks_eval = [(Pt, Pt_id) for Pt, Pt_id in zip(df_path_eval.Path, df_path_eval.Path_id)]
tasks_eval[0]

('D:\\isip.piconepress.com\\projects\\nedc\\data\\tuh_eeg\\tuh_eeg_abnormal\\v3.0.1\\edf\\eval\\abnormal\\01_tcp_ar\\aaaaabdo_s003_t000.edf',
 'campione_1_eval')

In [6]:
n_jobs = jb.cpu_count()
res = jb.Parallel(n_jobs=n_jobs, backend='loky')(jb.delayed(worker)(in_file, out_dir_eval, out_f_id) for in_file, out_f_id in tqdm(tasks_eval))

100%|██████████| 276/276 [09:00<00:00,  1.96s/it]


In [7]:
df_log = pd.DataFrame(res, columns=['path', 'status'])
df_log.to_csv(log_file_eval, index=False, encoding="utf-8-sig") 

In [5]:
n_jobs = jb.cpu_count()
res = jb.Parallel(n_jobs=n_jobs, backend='loky')(jb.delayed(worker)(in_file, out_dir_tr, out_f_id) for in_file, out_f_id in tqdm(tasks_tr))

100%|██████████| 2717/2717 [1:17:02<00:00,  1.70s/it]


KeyboardInterrupt: 

In [ ]:
df_log = pd.DataFrame(res, columns=['path', 'status'])
df_log.to_csv(log_file_tr, index=False, encoding="utf-8-sig") 

In [6]:
path_list_tr = u.get_path_list('D:/raw_data_tr', ['.npy'], True)
path_list_eval = u.get_path_list('D:/raw_data_eval', ['.npy'], True)

In [7]:
clips = []
chans = []
freqs = []
for path in tqdm(path_list_tr):
    file = np.load(path)
    clips.append(file.shape[0])
    chans.append(file.shape[1])
    freqs.append(file.shape[2])
print("number of clips: ", Counter(clips))
print("number of channels: ", Counter(chans))
print("different frequencies: ", Counter(freqs))
clips = []
chans = []
freqs = []
for path in tqdm(path_list_eval):
    file = np.load(path)
    clips.append(file.shape[0])
    chans.append(file.shape[1])
    freqs.append(file.shape[2])
print("number of clips: ", Counter(clips))
print("number of channels: ", Counter(chans))
print("different frequencies: ", Counter(freqs))

100%|██████████| 2713/2713 [16:30<00:00,  2.74it/s]


number of clips:  Counter({500: 2713})
number of channels:  Counter({6: 2713})
different frequencies:  Counter({1250: 2713})


100%|██████████| 276/276 [01:44<00:00,  2.64it/s]

number of clips:  Counter({500: 276})
number of channels:  Counter({6: 276})
different frequencies:  Counter({1250: 276})


In [2]:
files_directory_tr = "D:/raw_data_tr"
out_directory_tr = "D:/Dataset_tr"
dt.make_save_dataset(f_dir=files_directory_tr, out_dir=out_directory_tr, ratio=0.0)

paths presi
create liste di train e test dei path
create directory di test e train
inizio salvataggio file di train


100%|██████████| 2713/2713 [15:15<00:00,  2.96it/s]


(500, 6, 250)


MemoryError: Unable to allocate 7.58 GiB for an array with shape (1356500, 6, 250) and data type float32

In [2]:
files_directory_eval = "D:/raw_data_eval"
out_directory_eval = "D:/Dataset_eval"
dt.make_save_dataset(f_dir=files_directory_eval, out_dir=out_directory_eval, ratio=1.0)

paths presi
create liste di train e test dei path
create directory di test e train
inizio salvataggio file di test


100%|██████████| 276/276 [01:28<00:00,  3.13it/s]


(500, 6, 250)
save whole to D:/Dataset_eval\test
save delta to D:/Dataset_eval\test
save theta to D:/Dataset_eval\test
save alpha to D:/Dataset_eval\test
save low_beta to D:/Dataset_eval\test
save high_beta to D:/Dataset_eval\test
DONE!


In [6]:
from dataset import ClipDS
import torch
train_ds = ClipDS(data_dir='D:/Training_dataset/train',
                  band_name='whole',
                  clip_len=250)
train_loader = torch.utils.data.DataLoader(train_ds, shuffle=True, batch_size=16, drop_last=True, num_workers=0)

path:  D:/Training_dataset/train\whole.npy
elementi: 1196000
lunghezza campione: 250
lunghezza voluta clip: 250


In [19]:
for i in train_loader:
    print(i.shape)
    print(type(i))
    i = i.numpy()
    print(i.shape)
    print(type(i))
    print(i.shape)
    break

torch.Size([16, 1, 250])
<class 'torch.Tensor'>
(16, 1, 250)
<class 'numpy.ndarray'>
(16, 1, 250)


In [1]:
from sklearn.decomposition import PCA
import numpy as np